<a href="https://colab.research.google.com/github/TranPhu1999/HCMUS-NLPS-AIChallenge/blob/main/HCMUS_NLPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tham khảo
- Paper: https://paperswithcode.com/paper/frozen-in-time-a-joint-video-and-image
- Towee repo: https://github.com/towhee-io/towhee
- Video-text retrival: https://codelabs.towhee.io/how-to-build-a-text-video-retrieval-engine/index#1
- Similar video: https://codelabs.towhee.io/build-a-reverse-video-search-engine-in-minutes/index#0

# Load data
Download KeyFrames extracted from videos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !wget https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC00_V00.zip
# !unzip KeyFramesC00_V00.zip -d KeyFrames
# !rm -rf KeyFramesC00_V00.zip
# !wget https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC01_V00.zip
# !unzip KeyFramesC01_V00.zip -d KeyFrames
# !rm -rf KeyFramesC01_V00.zip
# !wget https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC02_V00.zip
# !unzip KeyFramesC02_V00.zip -d KeyFrames
# !rm -rf KeyFramesC02_V00.zip
# !wget https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC00_V01.zip
# !unzip KeyFramesC00_V01.zip -d KeyFrames
# !rm -rf KeyFramesC00_V01.zip
# !wget https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC01_V01.zip
# !unzip KeyFramesC01_V01.zip -d KeyFrames
# !rm -rf KeyFramesC01_V01.zip
# !wget https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC02_V01.zip
# !unzip KeyFramesC02_V01.zip -d KeyFrames
# !rm -rf KeyFramesC02_V01.zip

--2022-10-30 08:02:21--  https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC01_V00.zip
Resolving s3-north.viettelidc.com.vn (s3-north.viettelidc.com.vn)... 125.212.206.52, 125.212.206.48
Connecting to s3-north.viettelidc.com.vn (s3-north.viettelidc.com.vn)|125.212.206.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9207919383 (8.6G) [application/zip]
Saving to: ‘KeyFramesC01_V00.zip’

KeyFramesC01_V00.zi 100%[===================>]   8.58G  12.8MB/s    in 15m 43s 

2022-10-30 08:18:06 (9.31 MB/s) - ‘KeyFramesC01_V00.zip’ saved [9207919383/9207919383]

--2022-10-30 08:18:06--  https://s3-north.viettelidc.com.vn/aic2022-public-data/KeyFramesC02_V00.zip
Resolving s3-north.viettelidc.com.vn (s3-north.viettelidc.com.vn)... 125.212.206.48, 125.212.206.52
Connecting to s3-north.viettelidc.com.vn (s3-north.viettelidc.com.vn)|125.212.206.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11260966214 (10G) [application/zip]
Savi

In [ ]:
import os
import glob
print("Total number of images: {}".format(len(glob.glob("./KeyFrames/*/*.jpg"))))

226636

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/HCMUS_NLPS
%ll

/content/drive/MyDrive/Colab Notebooks/HCMUS_NLPS
total 14999
-rw------- 1 root 15353881 Oct 30 09:10 key_frame_df.csv
-rw------- 1 root      112 Oct 30 09:07 requirements.txt
drwx------ 2 root     4096 Oct 30 09:18 vi_en_translate/


In [ ]:
!pip install -r requirements.txt

In [ ]:
import towhee
import os
import glob
from PIL import Image
import pandas as pd
import numpy as np
import unicodedata

# Vietnamese Engslish Translate
Download pretrained model Vietnamese-English from VinAI repo

In [ ]:
try:
  os.mkdir("vi_en_translate")
except:
  pass

In [ ]:
%cd vi_en_translate
!wget https://huggingface.co/vinai/vinai-translate-vi2en/resolve/main/pytorch_model.bin
!wget https://huggingface.co/vinai/vinai-translate-vi2en/resolve/main/config.json
!wget https://huggingface.co/vinai/vinai-translate-vi2en/resolve/main/sentencepiece.bpe.model
!wget https://huggingface.co/vinai/vinai-translate-vi2en/resolve/main/tf_model.h5
%cd ..

/content/drive/MyDrive/Colab Notebooks/HCMUS_NLPS/vi_en_translate
--2022-10-30 09:16:31--  https://huggingface.co/vinai/vinai-translate-vi2en/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 52.3.185.208, 34.227.129.17, 2600:1f18:147f:e850:db19:5c51:ec6e:ddca, ...
Connecting to huggingface.co (huggingface.co)|52.3.185.208|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/ff/86/ff8651c4ac410bb102bd28be9525daee7edc5c7eb6f4eed159926ce0da08f1b8/245f08542f8b87d4d8721912ba3ae7d02feb5b72943d759135f4d551d4f32455?response-content-disposition=attachment%3B%20filename%3D%22pytorch_model.bin%22&Expires=1667380592&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZmLzg2L2ZmODY1MWM0YWM0MTBiYjEwMmJkMjhiZTk1MjVkYWVlN2VkYzVjN2ViNmY0ZWVkMTU5OTI2Y2UwZGEwOGYxYjgvMjQ1ZjA4NTQyZjhiODdkNGQ4NzIxOTEyYmEzYWU3ZDAyZmViNWI3Mjk0M2Q3NTkxMzVmNGQ1NTFkNGYzMjQ1NT9yZXNwb25zZS1jb250ZW50LWR

In [ ]:
dict_map = {
    "òa": "oà","Òa": "Oà","ÒA": "OÀ","óa": "oá","Óa": "Oá","ÓA": "OÁ","ỏa": "oả","Ỏa": "Oả","ỎA": "OẢ","õa": "oã",
    "Õa": "Oã","ÕA": "OÃ","ọa": "oạ","Ọa": "Oạ","ỌA": "OẠ","òe": "oè","Òe": "Oè","ÒE": "OÈ","óe": "oé","Óe": "Oé",
    "ÓE": "OÉ","ỏe": "oẻ","Ỏe": "Oẻ","ỎE": "OẺ","õe": "oẽ","Õe": "Oẽ","ÕE": "OẼ","ọe": "oẹ","Ọe": "Oẹ","ỌE": "OẸ",
    "ùy": "uỳ","Ùy": "Uỳ","ÙY": "UỲ","úy": "uý","Úy": "Uý","ÚY": "UÝ","ủy": "uỷ","Ủy": "Uỷ","ỦY": "UỶ","ũy": "uỹ",
    "Ũy": "Uỹ","ŨY": "UỸ","ụy": "uỵ","Ụy": "Uỵ","ỤY": "UỴ",
    }

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_vi2en = AutoTokenizer.from_pretrained("./vi_en_translate/", src_lang="vi_VN")
model_vi2en = AutoModelForSeq2SeqLM.from_pretrained("./vi_en_translate/")

def translate_vi2en(vi_text: str) -> str:
    for i, j in dict_map.items():
        vi_text = vi_text.replace(i, j)
    input_ids = tokenizer_vi2en(vi_text, return_tensors="pt").input_ids
    output_ids = model_vi2en.generate(
        input_ids,
        do_sample=True,
        top_k=100,
        top_p=0.8,
        decoder_start_token_id=tokenizer_vi2en.lang_code_to_id["en_XX"],
        num_return_sequences=1,
    )
    en_text = tokenizer_vi2en.batch_decode(output_ids, skip_special_tokens=True)
    en_text = " ".join(en_text)
    en_text = strip_accents(en_text)
    en_text = en_text.replace("\\","")
    return en_text

# vi_text = "Cô cho biết: trước giờ tôi không đến phòng tập công cộng, mà tập cùng giáo viên Yoga riêng hoặc tự tập ở nhà. Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn."
# print(translate_vi2en(vi_text))

# Towhee text-video inference
This section is only using example data, you can modfiy it to use your custom data

In [ ]:
!curl -L https://github.com/towhee-io/examples/releases/download/data/text_video_search.zip -O

In [ ]:
# Nếu chạy không được lệnh này thì ra ngoài folder rồi tự unzip
!unzip -q -o text_video_search.zip

In [ ]:
%ls

In [ ]:
import pandas as pd
import os

raw_video_path = './test_1k_compress' # 1k test video path.
test_csv_path = './MSRVTT_JSFUSION_test.csv' # 1k video caption csv.

test_sample_csv_path = './MSRVTT_JSFUSION_test_sample.csv'

sample_num = 1000 # you can change this sample_num to be smaller, so that this notebook will be faster.
test_df = pd.read_csv(test_csv_path)
print('length of all test set is {}'.format(len(test_df)))
sample_df = test_df.sample(sample_num, random_state=42)

sample_df['video_path'] = sample_df.apply(lambda x:os.path.join(raw_video_path, x['video_id']) + '.mp4', axis=1)

sample_df.to_csv(test_sample_csv_path)
print('random sample {} examples'.format(sample_num))

df = pd.read_csv(test_sample_csv_path)

df[['video_id', 'video_path', 'sentence']].head()

In [ ]:
from IPython import display
from pathlib import Path
import towhee
from PIL import Image

def display_gif(video_path_list, text_list):
    html = ''
    for video_path, text in zip(video_path_list, text_list):
        html_line = '<img src="{}"> {} <br/>'.format(video_path, text)
        html += html_line
    return display.HTML(html)

    
def convert_video2gif(video_path, output_gif_path, num_samples=16):
    frames = (
        towhee.glob(video_path)
              .video_decode.ffmpeg(sample_type='uniform_temporal_subsample', args={'num_samples': num_samples})
              .to_list()[0]
    )
    imgs = [Image.fromarray(frame) for frame in frames]
    imgs[0].save(fp=output_gif_path, format='GIF', append_images=imgs[1:], save_all=True, loop=0)


def display_gifs_from_video(video_path_list, text_list, tmpdirname = './tmp_gifs'):
    Path(tmpdirname).mkdir(exist_ok=True)
    gif_path_list = []
    for video_path in video_path_list:
        video_name = str(Path(video_path).name).split('.')[0]
        gif_path = Path(tmpdirname) / (video_name + '.gif')
        convert_video2gif(video_path, gif_path)
        gif_path_list.append(gif_path)
    return display_gif(gif_path_list, text_list)

In [ ]:
sample_show_df = sample_df[:3]
video_path_list = sample_show_df['video_path'].to_list()
text_list = sample_show_df['sentence'].to_list()
tmpdirname = './tmp_gifs'
display_gifs_from_video(video_path_list, text_list, tmpdirname=tmpdirname)

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.INT64, descrition='ids', is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='video retrieval')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2', #IP
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

collection = create_milvus_collection('text_video_retrieval', 512)

In [ ]:
import os
import towhee

device = 'cuda:0'
device = 'cpu'
# For the first time you run this line, 
# it will take some time 
# because towhee will download operator with weights on backend.
dc = (
    towhee.read_csv(test_sample_csv_path)
      .runas_op['video_id', 'id'](func=lambda x: int(x[-4:]))
      .video_decode.ffmpeg['video_path', 'frames'](sample_type='uniform_temporal_subsample', args={'num_samples': 12})
      .runas_op['frames', 'frames'](func=lambda x: [y for y in x])
      .video_text_embedding.clip4clip['frames', 'vec'](model_name='clip_vit_b32', modality='video')
      .to_milvus['id', 'vec'](collection=collection, batch=30)
)
print('Total number of inserted data is {}.'.format(collection.num_entities))

In [ ]:
device = 'cuda:0'
dc = (
    towhee.read_csv(test_sample_csv_path).unstream()
      .video_text_embedding.clip4clip['sentence','text_vec'](model_name='clip_vit_b32', modality='text')
      .milvus_search['text_vec', 'top10_raw_res'](collection='text_video_retrieval', limit=10)
      .runas_op['video_id', 'ground_truth'](func=lambda x : [int(x[-4:])])
      .runas_op['top10_raw_res', 'top1'](func=lambda res: [x.id for i, x in enumerate(res) if i < 1])
      .runas_op['top10_raw_res', 'top5'](func=lambda res: [x.id for i, x in enumerate(res) if i < 5])
      .runas_op['top10_raw_res', 'top10'](func=lambda res: [x.id for i, x in enumerate(res) if i < 10])
)

dc.select['video_id', 'sentence', 'ground_truth', 'top10_raw_res', 'top1', 'top5', 'top10']().show()

In [ ]:
benchmark = (
    dc.with_metrics(['mean_hit_ratio',])
        .evaluate['ground_truth', 'top1'](name='recall_at_1')
        .evaluate['ground_truth', 'top5'](name='recall_at_5')
        .evaluate['ground_truth', 'top10'](name='recall_at_10')
        .report()
)

In [ ]:
import gradio
import towhee.models.clip4clip

device = 'cuda:0'
collection='text_video_retrieval'
show_num = 3
with towhee.api() as api:
    milvus_search_function = (
         api.clip4clip(model_name='clip_vit_b32', modality='text', device=device)
            .milvus_search(collection=collection, limit=show_num)
            .runas_op(func=lambda res: [os.path.join(raw_video_path, 'video' + str(x.id) + '.mp4') for x in res])
            .as_function()
    )

interface = gradio.Interface(milvus_search_function, 
                             inputs=[gradio.Textbox()],
                             outputs=[gradio.Video(format='mp4') for _ in range(show_num)]
                            )

interface.launch(inline=True, share=True)

# Towhee text-image inference

In [ ]:
# for video in os.listdir("../AIChallenge_data/KeyFrames"):
#     fix_frame_df = pd.read_csv(os.path.join("../AIChallenge_data/Fix_KeyFrames/",video +".csv"),names=["img","frame"])
#     for i in range(len(fix_frame_df)):
#         img_path_df = img_path_df.append({"img_path":os.path.join("KeyFrames",video,fix_frame_df.iloc[i]["img"]),
#         "video_id":video,"frame":fix_frame_df.iloc[i]["frame"]},ignore_index=True)

In [ ]:
# img_path_df.to_csv("key_frame_df.csv")

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.INT64, descrition='ids', is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='text image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":512}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

In [ ]:
try:
  os.mkdir("KeyFrames")
except:
  pass

In [ ]:
!ln -s /content/KeyFrames .

In [ ]:
df_keyframes = pd.read_csv('key_frame_df.csv',index_col=0)
df_keyframes

,id,img_path,video_id,false_frame,frame
0,0,KeyFrames\C00_V0000\000000.jpg,C00_V0000,0,0
1,1,KeyFrames\C00_V0000\000096.jpg,C00_V0000,96,96
2,2,KeyFrames\C00_V0000\000206.jpg,C00_V0000,206,206
3,3,KeyFrames\C00_V0000\000378.jpg,C00_V0000,378,378
4,4,KeyFrames\C00_V0000\000545.jpg,C00_V0000,545,545
...,...,...,...,...,...
226631,226631,KeyFrames\C02_V0199\051029.jpg,C02_V0199,51029,51029
226632,226632,KeyFrames\C02_V0199\051040.jpg,C02_V0199,51040,51040
226633,226633,KeyFrames\C02_V0199\051200.jpg,C02_V0199,51200,51200
226634,226634,KeyFrames\C02_V0199\051360.jpg,C02_V0199,51360,51360


In [ ]:
import pandas as pd

import cv2
from towhee._types.image import Image

id_img = df_keyframes.set_index('id')['img_path'].to_dict()
def read_images(results):
    imgs = []
    for re in results:
        # print(re)
        path = id_img[re.id]
        imgs.append(path)
    return imgs

## Faiss local and colab

In [ ]:
# create image embeddings and build index
(
    towhee.read_csv('key_frame_df.csv')
          .runas_op['id', 'id'](func=lambda x: int(x))
          .set_parallel(4)
          .image_decode['img_path', 'img']()
          .image_text_embedding.clip['img', 'vec'](model_name='clip_vit_b32', modality='image')
          .tensor_normalize['vec','vec']()
          .to_faiss[('id', 'vec')](findex='./index.bin')
)

In [ ]:
queries = {}
with open("./query.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        query_name = line.split("//")[0]
        query = line.split("//")[1]
        queries[query_name] = query

## Milvus local machine only

In [ ]:
collection = create_milvus_collection('text_image_search', 512)
collection = Collection('text_image_challenge_search')
dc = (
    towhee.read_csv('key_frame_df.csv')
      .runas_op['id', 'id'](func=lambda x: int(x))
      .set_parallel(4)
      .image_decode['img_path', 'img']()
      .image_text_embedding.clip['img', 'vec'](model_name='clip_vit_b32', modality='image')
      .tensor_normalize['vec','vec']()
      .to_milvus['id', 'vec'](collection='text_image_search', batch=100)
      
)
print('Total number of inserted data is {}.'.format(collection.num_entities))

In [ ]:
for i in range(26,len(queries.keys())):
  key = list(queries.keys())[i]
  (
      towhee.dc['text']([queries[key]])
        .image_text_embedding.clip['text', 'vec'](model_name='clip_vit_b32', modality='text')
        .tensor_normalize['vec','vec']()
        .milvus_search['vec', 'result'](collection=collection, limit=100)
        .runas_op['result', 'result_img'](func=read_images)
        .select['result_img']()
        .to_csv(key+".csv")      
  )


# Evaluate query pack

In [ ]:
# !wget https://download039.fshare.vn/dl/hcL280KJz4ZEVrp8deuD4jastfEuPfRzeC8zbb7h6IC2mQKY7hd-pH7diN5uOkh2AbN4RiIRTjrXwh2C/query-pack-0.zip
# !wget https://download039.fshare.vn/dl/kQpla+QWiPVNJn72Xh4ZWETidn2Wt4oq7Uty-VfCscQA2pL8-kexe9VU6jYwCgcrENcyy47XaofvjKCc/query-pack-1.zip
# !wget https://download022.fshare.vn/dl/kWomGkkLX4UelE4fupVCUFlJmUkqiAVk56-42tZQ1e2+5VFjMMn78TTgvXzI7bElS1KnWpOw3EZ-JR+P/query-pack-2.zip

In [ ]:
# !unzip query-pack-0.zip -d query-pack
# !unzip query-pack-1.zip -d query-pack
# !unzip query-pack-2.zip -d query-pack

In [ ]:
# !rm -rf query-pack-0.zip
# !rm -rf query-pack-1.zip
# !rm -rf query-pack-2.zip

In [ ]:
# Only run this cell the first time you run this notebook to create a folder of english version of the queries
# Then you can comment this cell to only run the cell below to read that folder. 
query_dir = "D:\GitHub\AIChallenge_data\query-pack-0\query-pack-0"
en_query_dir = os.path.join(os.path.dirname(query_dir),"en_"+ query_dir.split("\\")[-1])
try:
    os.mkdir(en_query_dir)
except:
    pass
list_queries_file = os.listdir(query_dir)
en_queries = []
for query in list_queries_file:
    with open(os.path.join(query_dir,query),"r",encoding='utf8') as f:
        with open(os.path.join(en_query_dir,"en_" + query),"w",encoding='utf8') as f_out:
            read_line = f.readline()
            vi2en_text = translate_vi2en(read_line)
            f_out.write(vi2en_text)
            en_queries.append(vi2en_text)

en_queries

In [ ]:
query_dir = "D:\GitHub\AIChallenge_data\query-pack-0\en_query-pack-0"
list_queries_file = os.listdir(query_dir)
en_queries = []
for query in list_queries_file:
    with open(os.path.join(query_dir,query),"r",encoding='utf8') as f:
        read_line = f.readline()
        en_queries.append(read_line)

en_queries

['The news of a match of Vietnamese football team. Vietnamese players are celebrating after a goal. This is not the national team but the U19 team. The goal that lifted the score to 2-0 came from a close-fitting header. Opponents of Vietnam U19 are Brunei U19.',
 'A favorite recreational activity of young people. Flying kites in the suburbs of Ho Chi Minh City. In the picture there is a giant kite in the shape of a squid. On the same day was the launch of Chau Doc Bridge across the Hau parallel.',
 'A team car race, consisting of adults and children. These cars do not have engines, instead they are pushed by the parents. At the starting line, there are 3 people dressed up as zebras. This newscast was broadcast on the day that the day before was the Swiss National Day.',
 'A zoo in China with many different animals. In the frame there are 2 hippos. One of the 2 is drinking water. In addition to hippos, the zoo also has elephants and raccoons.',
 'A picture of U.S. President Joe Biden. T

In [ ]:
collection = Collection("text_image_search")
(
      towhee.dc['text'](en_queries)
        .image_text_embedding.clip['text', 'vec'](model_name='clip_vit_b32', modality='text')
        .tensor_normalize['vec','vec']()
        .milvus_search['vec', 'result'](collection=collection, limit=100)
        .runas_op['result', 'result_img'](func=read_images)
        .select['text','result_img']()
        .to_csv("test.csv")
)

In [ ]:
# The result will be save in a folder name "result" in the folder contain queries pack
from ast import literal_eval
df = pd.read_csv("test.csv")
df.result_img =df.result_img.apply(literal_eval)

result_query_dir = os.path.join(os.path.dirname(query_dir),"result")
try:
    os.mkdir(result_query_dir)
except:
    pass

for i in range(len(list_queries_file)):
    with open(os.path.join(result_query_dir,"{}.csv".format(list_queries_file[i].split(".")[0].replace("en_",""))),"w") as f:
        list_img = df["result_img"][i]
        for img in list_img:
            video_id = img.split("\\")[-2]
            false_frame = img.split("\\")[-1].split(".")[0]
            real_frame = df_keyframes[df_keyframes.false_frame == int(false_frame)][video_id==df_keyframes.video_id].frame.values[0]
            f.write(video_id + ".mp4, " + str(false_frame) + '\n')

# Calculate score

In [ ]:
import numpy as np
# Use this link if it's still available to check the queries results
# Link: http://128.199.178.135:4567/?fbclid=IwAR0rtcLToubngq62z765Ee-gp-2Wxl_BEp-cASrUkEo1ujn7ZnZLjoRFRXE
# put your rank of the 10 queries here
#----------------------------------------
rank = [18,5,1,1,2,33,101,63,1,13]
#----------------------------------------

accept_rank = [1,5,20,50,100]
score = []
for i in range(len(rank)):
    for j in range(len(accept_rank)):
        score.append(1 if rank[i] <= accept_rank[j] else 0)

total_score = np.sum(score) / 5
total_score


6.4